In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle
from somhos.config.paths import get_relative_path
from somhos.config.paths import DOCS_SAMPLE_A_SUFFIX, DOCS_SAMPLE_B_SUFFIX
from somhos.config.paths import DOCS_SAMPLES_CONTENT, SAMPLE_PATH, DOCS_SAMPLES_WORD_COUNT
from somhos.config.paths import DOCS_SAMPLES_WORD_DOC_COUNT
from somhos.config.paths import DOCS_SAMPLES_JACCARD_SIM, DOCS_SAMPLES_JACCARD_SIM_UDV
from somhos.config.paths import DOCS_SAMPLES_TF_IDF, KPS_DIRECTORY_INVERSE_SUFFIX

Default path

In [2]:
data_path = "../../src/somhos/resources/aminer/v9"

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (1000, 1000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 2000


In [5]:
from nltk.metrics.distance import jaccard_distance
from itertools import product
import numpy as np

In [6]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_bow(index, samples_content):
    return samples_content[index]['bag-of-words']

m_jaccard_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    set_a = get_bow(docs_sample_a[i], samples_content)
    set_b = get_bow(docs_sample_b[j], samples_content)
    jaccard_sim = 1 - jaccard_distance(set_a, set_b)
    m_jaccard_sim[i, j] = jaccard_sim

print(m_jaccard_sim)

[[0.03292181 0.03518519 0.00714286 ... 0.04385965 0.02083333 0.04580153]
 [0.04471545 0.0291439  0.02797203 ... 0.05627706 0.00657895 0.03623188]
 [0.03508772 0.02448211 0.008      ... 0.04205607 0.04761905 0.05172414]
 ...
 [0.03508772 0.01492537 0.02439024 ... 0.04205607 0.03125    0.02521008]
 [0.0361991  0.03468208 0.01709402 ... 0.04854369 0.02459016 0.        ]
 [0.05172414 0.01845018 0.         ... 0.03125    0.02189781 0.03174603]]


In [7]:
docs_samples_jaccard_sim_path = get_relative_path(data_path, DOCS_SAMPLES_JACCARD_SIM)
save_pickle(m_jaccard_sim, docs_samples_jaccard_sim_path)

In [8]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_jaccard_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_jaccard_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_jaccard_sim)

[[0.04361271 0.0460296  0.02870448 ... 0.04959599 0.02688767 0.03650811]
 [0.03771801 0.03980823 0.02482477 ... 0.04289259 0.02325353 0.03157367]
 [0.04338416 0.04578839 0.02855405 ... 0.04933609 0.02674677 0.03631679]
 ...
 [0.03332177 0.03516837 0.02193131 ... 0.03789323 0.02054321 0.02789359]
 [0.03247137 0.03427084 0.02137161 ... 0.03692616 0.02001893 0.02718172]
 [0.03512217 0.03706854 0.02311628 ... 0.03994062 0.02165317 0.02940069]]


In [9]:
docs_samples_jaccard_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_JACCARD_SIM_UDV)
save_pickle(m_udv_jaccard_sim, docs_samples_jaccard_sim_udv_path)